In [ ]:
import pandas as pd
import numpy as np
import string

In [ ]:
def make_ids(size: float=1e2, length: int=8):
    size = int(size)
    space = string.ascii_lowercase+string.digits
    out = set()
    for _ in range(2*size):
        arr = np.random.choice(list(space), size=length)
        val = ''.join(arr)
        out.add(val)
    return list(out)[:size]

In [ ]:
def make_data(size: float=1e2) -> pd.DataFrame:
    size = int(size)
    years = np.arange(2012, 2017).astype(str)
    df = pd.DataFrame(dict(
        id=np.random.choice(make_ids(8), size=size),
        year=np.random.choice(years, size=size),
        month=np.random.randint(1, 13, size=size),
        imp=np.random.randint(100, size=size)
    ), columns=('id', 'year', 'month', 'quarter', 'imp'))
    df.quarter = df.month.apply(lambda m: int((m-1)/3+1))
    for label in ['id', 'year', 'month', 'quarter']:
        df[label] = df[label].astype('category')
    return df

In [ ]:
data = make_data(size=1e4)
data.head()

In [ ]:
groupby_cols = ['id', 'year', 'quarter']

In [ ]:
%%time
lookup = data.groupby(groupby_cols).mean()

In [ ]:
lookup.head()

In [ ]:
qid = lookup.index.get_level_values('id')[0]
lookup.loc[qid, '2012']

In [ ]:
%%time
lookup_dict = {}
for qid in data.id.cat.categories:
    frame = lookup.loc[qid].imp
    lookup_dict[qid] = pd.DataFrame(dict(
        x=frame,
        s1=frame.shift(-1),
        s2=frame.shift(-2)
    ), columns=('x', 's1', 's2'))

In [ ]:
for k, tab in lookup_dict.items():
    print(k)
    print(tab.dropna())
    break

In [ ]:
%%time
out = pd.DataFrame()
for name, group in data.groupby(groupby_cols):
    qid, year, quarter = name
    part = lookup_dict[qid].loc[year, quarter]
    group = group.copy()
    group['x'] = part.x
    group['s1'] = part.s1
    group['s2'] = part.s2
    group = group.dropna()
    out = out.append(group)

In [ ]:
out.head()

In [ ]:
%%time
rlookup = {}
for name, group in data.loc[:, groupby_cols].groupby(groupby_cols):
    qid, year, quarter = name
    part = lookup_dict[qid].loc[year, quarter]
    for i in group.index:
        rlookup[i] = (part.x, part.s1, part.s2)

In [ ]:
%%time
rdf = pd.DataFrame.from_dict(rlookup, orient='index')
rdf.columns = ['x', 's1', 's2']

In [ ]:
rdf.head()

In [ ]:
%%time
for l in ['x', 's1', 's2']:
    data.loc[:, l] = rdf.loc[:, l]
    del rdf[l]

In [ ]:
data.dropna()